# Проект по SQL: Книги по подписке

**Описание проекта**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

Таблица **books** содержит данные о книгах:

- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица **authors** содержит данные об авторах:

- author_id — идентификатор автора;
- author — имя автора.

Таблица **publishers** содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства.

Таблица **ratings** содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица **reviews** содержит данные о пользовательских обзорах:

- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

[Схема данных](https://disk.yandex.ru/i/YJanXmVOmojr1w)

**Цели исследования**

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

# Подключение к базе данных

Загрузим библиотеки.

In [2]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import text, create_engine
from IPython.display import Image

In [3]:
!pip install psycopg2

Установим параметры и сохраним коннектор.

In [4]:
db_config = {
            'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db' # название базы данных
            }
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [5]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [6]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Исследуем данные

In [7]:
Image(url='https://github.com/kurenovk/YP_Books_SQL/blob/main/DATA/data_scheme.png?raw=true')

Зададим функцию для выполнения SQL-запроса с помощью Pandas.

In [8]:
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

Для каждой таблицы выведем первые 5 строк, количество строк и общую информацию.

Таблица `books`:

In [9]:
query = ''' SELECT *
            FROM books LIMIT 5 '''

get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Посчитаем количество строк в таблице `books`.

In [18]:
query = ''' SELECT COUNT(*)
            FROM books ''' 

get_sql_data(query)

,count
0,1000


Выведем информацию о таблице `books`.

In [19]:
query = ''' SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = 'books' ''' 

get_sql_data(query)

,column_name,data_type,is_nullable
0,book_id,integer,NO
1,author_id,integer,YES
2,num_pages,integer,YES
3,publication_date,date,YES
4,publisher_id,integer,YES
5,title,text,YES


Таблица `books` содержит 6 столбцов и 1000 строк. Столбцы `book_id`, `author_id`, `num_pages` и `publisher_id` имеют тип данныйх `int`. Столбец `publication_date` имеет тип данных `date`. Столбец `title` имеет тип данных `text`.

Таблица `authors`:

In [20]:
query = ''' SELECT * 
            FROM authors LIMIT 5 '''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Посчитаем количество строк в таблице `authors`.

In [21]:
query = ''' SELECT COUNT(*)
            FROM authors ''' 

get_sql_data(query)

,count
0,636


Выведем информацию о таблице `authors`.

In [22]:
query = ''' SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = 'authors' ''' 

get_sql_data(query)

,column_name,data_type,is_nullable
0,author_id,integer,NO
1,author,text,YES


Таблица `authors` содержит 2 столбца и 636 строк. Столбец `author_id` имеет тип данныйх `int`. Столбец `author` имеет тип данных `text`.

Таблица `publishers`:

In [23]:
query = ''' SELECT * 
            FROM publishers LIMIT 5 '''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Посчитаем количество строк в таблице `publishers`.

In [24]:
query = ''' SELECT COUNT(*)
            FROM publishers ''' 

get_sql_data(query)

,count
0,340


Выведем информацию о таблице `publishers`.

In [25]:
query = ''' SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = 'publishers' ''' 

get_sql_data(query)

,column_name,data_type,is_nullable
0,publisher_id,integer,NO
1,publisher,text,YES


Таблица `publishers` содержит 2 столбца и 340 строк. Столбец `publisher_id` имеет тип данныйх `int`. Столбец `publisher` имеет тип данных `text`.

Таблица `ratings`:

In [26]:
query = ''' SELECT * 
            FROM ratings LIMIT 5 '''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Посчитаем количество строк в таблице `ratings`.

In [27]:
query = ''' SELECT COUNT(*)
            FROM ratings ''' 

get_sql_data(query)

,count
0,6456


Выведем информацию о таблице `ratings`.

In [28]:
query = ''' SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = 'ratings' ''' 

get_sql_data(query)

,column_name,data_type,is_nullable
0,rating_id,integer,NO
1,book_id,integer,YES
2,rating,integer,YES
3,username,text,YES


Таблица `ratings` содержит 4 столбца и 6456 строк. Столбцы `rating_id`, `book_id`, и `rating` имеют тип данныйх `int`. Столбец `username` имеет тип данных `text`.

Таблица `reviews`:

In [29]:
query = ''' SELECT * 
            FROM reviews LIMIT 5 '''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Посчитаем количество строк в таблице `reviews`.

In [30]:
query = ''' SELECT COUNT(*)
            FROM reviews ''' 

get_sql_data(query)

,count
0,2793


Выведем информацию о таблице `reviews`.

In [31]:
query = ''' SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = 'reviews' ''' 

get_sql_data(query)

,column_name,data_type,is_nullable
0,review_id,integer,NO
1,book_id,integer,YES
2,username,text,YES
3,text,text,YES


Таблица `ratings` содержит 4 столбца и 2793 строки. Столбцы `review_id` и `book_id` имеют тип данныйх `int`. Столбцы `username` и `text` имеют тип данных `text`.

# Количество книг, вышедших после 1 января 2000 года

In [32]:
query = ''' SELECT COUNT(book_id) 
            FROM books
            WHERE publication_date > '2000-01-01' '''

get_sql_data(query)

,count
0,819


После 1 января 2000 года вышло 819 книг.

# Количество обзоров и средняя оценка для каждой книги

In [33]:
query = ''' SELECT title,
                   COUNT(DISTINCT review_id) AS reviews_count,
                   AVG(rating) AS rating_avg
            FROM books
                 LEFT JOIN reviews USING(book_id)
                 LEFT JOIN ratings USING(book_id)
            GROUP BY book_id '''

get_sql_data(query)

,title,reviews_count,rating_avg
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


# Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [34]:
query = ''' SELECT publisher
            FROM publishers
                 JOIN books USING(publisher_id)
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY COUNT(book_id) DESC
            LIMIT 1 '''

get_sql_data(query)

,publisher
0,Penguin Books


Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц.

# Автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [10]:
query = '''
SELECT 
    a.author,
    AVG(avg_r.avg_rating) AS average_rating
FROM 
    (SELECT 
         b.author_id,
         AVG(ra.rating) AS avg_rating
     FROM 
         books AS b
         LEFT JOIN ratings AS ra ON ra.book_id = b.book_id
     GROUP BY 
         b.author_id, b.book_id
     HAVING 
         COUNT(ra.rating_id) >= 50) AS avg_r
JOIN 
    authors AS a ON a.author_id = avg_r.author_id
GROUP BY 
    a.author
ORDER BY 
    average_rating DESC
LIMIT 1;

'''
get_sql_data(query)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автором с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, является J.K. Rowling/Mary GrandPré со средней оценкой книг 4.41.

# Среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [36]:
query = ''' WITH active_critics AS (SELECT DISTINCT username,
                                           COUNT(DISTINCT review_id) AS reviews_count,
                                           COUNT(DISTINCT rating_id) AS ratings_count
                                    FROM reviews JOIN ratings USING(username)
                                    GROUP BY 1
                                    HAVING COUNT(DISTINCT rating_id) > 48
                                    ORDER BY 3)
            SELECT AVG(reviews_count)
            FROM active_critics '''

get_sql_data(query)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок равно 24.

### Выводы

- После 1 января 2000 года вышло 819 книг;
- Для каждой книги было посчитано количество обзоров и средняя оценка;
- Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц;
- Автором с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками, является J.K. Rowling/Mary GrandPré со средней оценкой книг 4.41;
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок равно 24.

В качестве варианта ценностного предложения для нового продукта можно добавить в сервис функцию рекомендации книг читателям, основанную на популярности книги и автора, а также на рейтинге и количестве обзоров.